- Fine-tune multi-lingual - with both Hindi and Nepali
- Fine tune with Hindi run inference on Nepali 
- Fine tune with Nepali run inference on Hindi
- Fine tune with Nepali run inference on Nepali
- Fine tune with Hindi run inference on Hindi

In [1]:
from kaggle_secrets import UserSecretsClient


In [2]:
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("GITHUB_KEY")

In [3]:
!git clone https://{token}@github.com/shreeya-dhakal/llama-3-finetune.git

Cloning into 'llama-3-finetune'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 137 (delta 69), reused 73 (delta 19), pack-reused 0
Receiving objects: 100% (137/137), 27.97 MiB | 18.17 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [4]:
%%capture
!mamba install --force-reinstall aiohttp -y
!pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# Temporary fix for https://github.com/huggingface/datasets/issues/6753
!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

In [5]:
import os
import wandb

os.environ["WANDB_PROJECT"]="llama-finetune-nepali-alpaca" # Change this
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
wandb_token = user_secrets.get_secret("WANDB_KEY")
wandb.login(key= wandb_token)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
from datasets import load_dataset, load_metric, concatenate_datasets
import sys
import os
sys.path.append('/kaggle/working/llama-3-finetune/src/')
from data.data_sampler import sample_data
from unsloth import FastLanguageModel
import torch
import re

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-07-19 03:21:46.816518: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-19 03:21:46.816608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-19 03:21:46.997396: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
split_ratio = 0.2
seed = 42

In [9]:
# dataset_name = "CohereForAI/aya_dataset"
# aya_dataset = load_dataset(dataset_name)
# dataset_type = "aya"

Generating train split:   0%|          | 0/202362 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1750 [00:00<?, ? examples/s]

In [10]:
# lang_multi = "Nepali"
# aya_nepali_train, _ = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)
# aya_nepali_train

Filter:   0%|          | 0/202362 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1750 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4002 [00:00<?, ? examples/s]

Dataset({
    features: ['inputs', 'targets', 'language', 'language_code', 'annotation_type', 'user_id'],
    num_rows: 3201
})

In [11]:
# lang_multi = "Hindi"
# aya_hindi_train, _ = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)
# aya_hindi_train

Filter:   0%|          | 0/202362 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1750 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1153 [00:00<?, ? examples/s]

Dataset({
    features: ['inputs', 'targets', 'language', 'language_code', 'annotation_type', 'user_id'],
    num_rows: 922
})

In [12]:
# aya_nepali_train = aya_nepali_train.rename_columns(
#             {"inputs": "input", "targets": "output"})
# new_column_data = [""] * len(aya_nepali_train)  # Create a list with the same length as the dataset
# aya_nepali_train = aya_nepali_train.add_column("instruction", new_column_data)
# aya_nepali_train = aya_nepali_train.remove_columns(['language', 'language_code', 'annotation_type', 'user_id'])
# aya_nepali_train

Flattening the indices:   0%|          | 0/3201 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 3201
})

In [13]:
# aya_hindi_train = aya_hindi_train.rename_columns(
#             {"inputs": "input", "targets": "output"})
# new_column_data = [""] * len(aya_hindi_train)
# aya_hindi_train = aya_hindi_train.add_column("instruction", new_column_data)
# aya_hindi_train = aya_hindi_train.remove_columns(['language', 'language_code', 'annotation_type', 'user_id'])
# aya_hindi_train

Flattening the indices:   0%|          | 0/922 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 922
})

In [8]:
# dataset_name = "Saugatkafley/alpaca-nepali-sft"

In [9]:
# alpaca_nepali_train, _ = sample_data(dataset_name, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)
# alpaca_nepali_train

Generating train split:   0%|          | 0/52005 [00:00<?, ? examples/s]

Filter:   0%|          | 0/52005 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51963 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51963 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'id'],
    num_rows: 41567
})

In [16]:
dataset_name = "iamshnoo/alpaca-cleaned-hindi"

In [17]:
alpaca_hindi_train, _ = sample_data(dataset_name, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)
alpaca_hindi_train

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'instruction', 'output'],
    num_rows: 41408
})

In [18]:
# nepali_train = concatenate_datasets([aya_nepali_train, alpaca_nepali_train])
# nepali_train = nepali_train.shuffle(seed=42)
# nepali_train

Dataset({
    features: ['input', 'output', 'instruction', 'id'],
    num_rows: 44768
})

In [19]:
# hindi_train = concatenate_datasets([aya_hindi_train, alpaca_hindi_train])
# hindi_train = hindi_train.shuffle(seed=42)
# hindi_train

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 42330
})

In [19]:
# both_train = concatenate_datasets([nepali_train, hindi_train])
# both_train = both_train.shuffle(seed=42)
# both_train

In [10]:
# #Nepali
# prompt = """Below is an instruction in Nepali that describes a task, paired with an input also in Nepali that provides further context. Write a response in Nepali that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

In [ ]:
#Hindi
prompt = """Below is an instruction in Hindi that describes a task, paired with an input also in Hindi that provides further context. Write a response in Hindi that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [11]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

tokenizer.add_special_tokens({"pad_token": "<|reserved_special_token_0|>"})
model.config.pad_token_id = tokenizer.pad_token_id
tokenizer.padding_side = 'right'

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.25.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

In [12]:
print(f"Pad Token id: {tokenizer.pad_token_id} and Pad Token: {tokenizer.pad_token}")
print(f"EOS Token id: {tokenizer.eos_token_id} and EOS Token: {tokenizer.eos_token}")

Pad Token id: 128002 and Pad Token: <|reserved_special_token_0|>
EOS Token id: 128001 and EOS Token: <|end_of_text|>


In [13]:
# LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [14]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

train_data = alpaca_nepali_train #Change it to Hindi
train_data = train_data.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/41567 [00:00<?, ? examples/s]

In [64]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_data,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 100,
        save_steps=500,                 ### Checkpoint will be save after every 500 steps
        optim = "adamw_8bit",
#         num_train_epochs=2,
        max_steps = 10392,
        weight_decay = 0.01,
        report_to="wandb", 
        run_name="llama3_nep_alpaca_0714", 
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs_alpaca_nep_0714",   # Saving the checkpoints to outputs folder
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#train_data = 10,392
trainer_stats = trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 41,567 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 10,392
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
100,0.808700
200,0.976700
300,0.945900


In [ ]:
wandb.finish()

In [15]:
# import wandb
# run = wandb.init()
# artifact = run.use_artifact('icodeformybhasa/llama-finetune-nepali-alpaca/model-lively-wildflower-10:v9', type='model')
# artifact_dir = artifact.download()

wandb: Currently logged in as: ssdhakal57 (icodeformybhasa). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-lively-wildflower-10:v9, 250.58MB. 11 files... 
wandb:   11 of 11 files downloaded.  
Done. 0:0:2.0


In [16]:
# import shutil
# import os

# def copy_files(src_dir, dst_dir):

#     os.makedirs(dst_dir, exist_ok=True)

#     file_names = os.listdir(src_dir)

#     if not os.listdir(dst_dir):
#         for file_name in file_names:
#             src_file = os.path.join(src_dir, file_name)
#             dst_file = os.path.join(dst_dir, file_name)
#             shutil.move(src_file, dst_file)
#     else:
#         print("Files have already been moved")

#     try:
#         os.rmdir(src_dir)
#     except OSError:
#         shutil.rmtree(src_dir)

# src_dir = artifact_dir
# dst_dir = "/kaggle/working/outputs/checkpoint-9900"  

# copy_files(src_dir, dst_dir)

In [17]:
# from trl import SFTTrainer
# from transformers import TrainingArguments

# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = train_data,
#     dataset_text_field = "text",
#     max_seq_length = max_seq_length,
#     dataset_num_proc = 2,
#     packing = False, # Can make training 5x faster for short sequences.
#     args = TrainingArguments(
#         per_device_train_batch_size = 2,
#         gradient_accumulation_steps = 4,
#         warmup_steps = 5,
#         learning_rate = 2e-4,
#         fp16 = not torch.cuda.is_bf16_supported(),
#         bf16 = torch.cuda.is_bf16_supported(),
#         logging_steps = 100,
#         save_steps=500,                 ### Checkpoint will be save after every 500 steps
#         optim = "adamw_8bit",
# #         num_train_epochs=2,
#         max_steps = 10392,
#         weight_decay = 0.01,
#         report_to="wandb", 
#         run_name="llama3_nep_alpaca_0714_9900", 
#         lr_scheduler_type = "linear",
#         seed = 3407,
#         output_dir = "outputs",
#     ),
# )

Map (num_proc=2):   0%|          | 0/41567 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [18]:
# trainer_stats = trainer.train(resume_from_checkpoint = True)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 41,567 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 10,392
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
9600,0.611200
9700,0.618400
9800,0.627700
9900,0.619000
10000,0.605300
10100,0.607200
10200,0.600700
10300,0.605400


wandb: Adding directory to artifact (./outputs/checkpoint-10000)... Done. 0.8s
wandb: Adding directory to artifact (./outputs/checkpoint-10392)... Done. 0.8s
max_steps is given, it will override any value given in num_train_epochs


In [20]:
# wandb.finish()

In [21]:
# hf_token = user_secrets.get_secret("HF_HUB")

In [41]:
# # Save LoRA
# model.push_to_hub("shreeyad/nep_llama-3_lora_model", token = hf_token)
# tokenizer.push_to_hub("shreeyad/nep_llama-3_lora_model", token = hf_token)

README.md:   0%|          | 0.00/575 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/shreeyad/nep_llama-3_lora_model


In [79]:
# import tempfile

# with tempfile.TemporaryDirectory() as tmp_dir:
#     model.save_pretrained(tmp_dir)
#     model.push_to_hub_merged("shreeyad/nep_llama-3_16bit_model", tokenizer, save_method = "merged_16bit", token = hf_token, use_temp_dir=False)
#This runs out of space in Kaggle so we will use colab to merge LoRA to the base model

In [78]:
# model.push_to_hub_merged("shreeyad/nep_llama-3_4bit_model", tokenizer, save_method = "merged_4bit_forced", token = hf_token)

Unsloth: Merging 4bit and LoRA weights to 4bit...
This might take 5 minutes...


/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Done.
Unsloth: Saving 4bit Bitsandbytes model. Please wait...


README.md:   0%|          | 0.00/575 [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/581 [00:00<?, ?B/s]

Saved merged_4bit model to https://huggingface.co/shreeyad/nep_llama-3_4bit_model


In [77]:
# !rm -rf _unsloth_temporary_saved_buffers

In [76]:
# !du -h --max-depth=1 /kaggle/working

14G	/kaggle/working/_unsloth_temporary_saved_buffers
6.6G	/kaggle/working/nep_llama-3_16bit_model
20G	/kaggle/working
